# Omaha Incidents

## Data Exploration

Let"s explore the data a little bit to see what kind of analysis and visualizations we want to implement.

In [ ]:
!pip3 install ipykernel
!pip3 install --upgrade pandas plotly dash "nbformat>=4.2.0"

In [ ]:
import pandas as pd
import sqlite3

In [ ]:
connection = sqlite3.connect("../raw_data/ingress.db")

In [ ]:
cursor = connection.cursor()

In [ ]:
# Test query to see if the data loaded
select_all = "SELECT * FROM incidents;"

In [ ]:
df = pd.read_sql_query(select_all, connection)
df.head()

In [ ]:
# test plotting by sorting & plotting top 5 crime categories
s = df.value_counts(subset=["description"])
t = s.nlargest(5)
t.head()
t.plot(kind="bar", title="Top 5 Incident Categories")

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
s.head(10)

In [ ]:
filtered_df = df[(df['date'] > '01/01/2023') & (df['date'] < '12/31/2023')]
filtered_df.head()

In [ ]:
fig = px.scatter_mapbox(
    filtered_df,
    lat="lat",
    lon="lon",
    color="description",
    hover_name="description",
    hover_data=["date", "time"],
    title="Incident Count by Coordinates",
    center={"lat": 41.257160, "lon": -95.995102},
    zoom=10
)

# fig.update_layout(showlegend=False)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.update_layout(mapbox_bounds={"west": -180, "east": -50, "south": 20, "north": 90})
# fig.show()

In [ ]:
import plotly.io as pio
pio.write_html(fig, file="test.html", auto_open=False)

In [ ]:
# clean up and close it out
connection.commit()
connection.close()